In [ ]:
import flopy as fp
import numpy as np
import pandas as pd
import shutil

In [ ]:
ctl_filename = 'freyberg_delta.ctl'

In [ ]:
sim = fp.mf6.MFSimulation.load('mfsim.nam', sim_ws='freyberg_demo')
m = sim.get_model(model_name='freyberg')

In [ ]:
nrow,ncol = m.dis.nrow.get_data(), m.dis.ncol.get_data()
resolution = m.dis.delr.get_data()[0]

## Let's make a control file. This is basically hacked from the SWB2 Central Sands model
We are going to make some dictionaries with the information to write out later on

In [ ]:
xll,yll=516402.147,1172605.575
proj4 = '+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=23 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs'
prof4_daymet = '+proj=lcc +lat_1=25.0 +lat_2=60.0 +lat_0=42.5 +lon_0=-100.0 +x_0=0.0 +y_0=0.0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs'
grid_info = {'GRID': '{0:d} {1:d} {2:f} {3:f} {4:f} '.format(ncol,nrow, xll, yll, resolution),
            'BASE_PROJECTION_DEFINITION': proj4}

In [ ]:
methods = { 'INTERCEPTION_METHOD':              'BUCKET',
            'EVAPOTRANSPIRATION_METHOD':        'HARGREAVES',
            'RUNOFF_METHOD':                    'CURVE_NUMBER',
            'SOIL_MOISTURE_METHOD':             'FAO-56_TWO_STAGE',
            'PRECIPITATION_METHOD':             'GRIDDED',
            'GROWING_DEGREE_DAY_METHOD':        'BASKERVILLE_EMIN',
            'FOG_METHOD':                       'NONE',
            'FLOW_ROUTING_METHOD':              'NONE',
            'IRRIGATION_METHOD':                'FAO-56',
            'ROOTING_DEPTH_METHOD' :            'DYNAMIC',
            'CROP_COEFFICIENT_METHOD':          'FAO-56',
            'DIRECT_RECHARGE_METHOD':           'NONE',
            'SOIL_STORAGE_MAX_METHOD':          'CALCULATE',
            'AVAILABLE_WATER_CONTENT_METHOD' :  'GRIDDED'}

In [ ]:
grids = {'HYDROLOGIC_SOILS_GROUP': 'ARC_GRID swb_files/soil_group.asc',
        'HYDROLOGIC_SOILS_GROUP_PROJECTION_DEFINITION': proj4,
        'LAND_USE': 'ARC_GRID swb_files/land_use.asc',
        'LANDUSE_PROJECTION_DEFINITION': proj4,
        'AVAILABLE_WATER_CONTENT': 'ARC_GRID swb_files/available_water_content.asc',
        'AVAILABLE_WATER_CONTENT_PROJECTION_DEFINITION': proj4,
        'IRRIGATION_MASK': 'ARC_GRID swb_files/irrigation.asc',
        'IRRIGATION_MASK_PROJECTION_DEFINITION': proj4}

In [ ]:
weather_data = {
    'PRECIPITATION': 'NETCDF datasets/daily_weather/daymet_v3_prcp_%y_na__MAP_Model_Subset.nc4',
    'PRECIPITATION_GRID_PROJECTION_DEFINITION': prof4_daymet,
    'PRECIPITATION_NETCDF_Z_VAR':                'prcp',
    'PRECIPITATION_SCALE_FACTOR':          0.03937008,
    'PRECIPITATION_MISSING_VALUES_CODE':      -9999.0,
    'PRECIPITATION_MISSING_VALUES_OPERATOR':      '<=',
    'PRECIPITATION_MISSING_VALUES_ACTION':       'zero',

    'TMAX': 'NETCDF datasets/daily_weather/daymet_v3_tmax_%y_na__MAP_Model_Subset.nc4',
    'TMAX_GRID_PROJECTION_DEFINITION': prof4_daymet,
    'TMAX_SCALE_FACTOR':                 1.8,
    'TMAX_ADD_OFFSET':                  32.0,
    'TMAX_MISSING_VALUES_CODE':      -9999.0,
    'TMAX_MISSING_VALUES_OPERATOR':      '<=',
    'TMAX_MISSING_VALUES_ACTION':       'mean',

    'TMIN': 'NETCDF datasets/daily_weather/daymet_v3_tmin_%y_na__MAP_Model_Subset.nc4',
    'TMIN_GRID_PROJECTION_DEFINITION': prof4_daymet,
    'TMIN_SCALE_FACTOR':                 1.8,
    'TMIN_ADD_OFFSET':                  32.0,
    'TMIN_MISSING_VALUES_CODE':      -9999.0,
    'TMIN_MISSING_VALUES_OPERATOR':      '<=',
    'TMIN_MISSING_VALUES_ACTION':       'mean',

    'INITIAL_CONTINUOUS_FROZEN_GROUND_INDEX': 'CONSTANT 25.0',
    'UPPER_LIMIT_CFGI': '83.',
    'LOWER_LIMIT_CFGI': '55.'
}

In [ ]:
lookup_tables = {
    'LAND_USE_LOOKUP_TABLE': 'swb_files/Landuse_lookup_CDL.txt',
    'IRRIGATION_LOOKUP_TABLE': 'swb_files/Irrigation_lookup_CDL.txt'
}

In [ ]:
general_directives = {
    'INITIAL_PERCENT_SOIL_MOISTURE': 'CONSTANT 100.0',
    'INITIAL_SNOW_COVER_STORAGE': 'CONSTANT 0.0',
    'START_DATE': '01/01/2010',
    'END_DATE': '12/31/2017'
}

In [ ]:
def arcify_rasters(fname, xll, yll, nrow, ncol, resolution):
    indat = np.loadtxt(fname)
    np.savetxt(fname.replace('.dat','.asc'), indat, comments='',
               header='ncols {0:d}\nnrows {1:d}\nxllcorner {2:f}\nyllcorner {3:f}\ncellsize {4:f}\nNODATA_value 0'.format(
               ncol, nrow, xll, yll, resolution),
              fmt = '%d',
              )
    shutil.copy2('swb_files/5070.prj', fname.replace('.dat','.prj'))

In [ ]:
arcify_rasters('swb_files/land_use.dat', xll, yll, nrow, ncol, resolution)
arcify_rasters('swb_files/parcels.dat', xll, yll, nrow, ncol, resolution)
arcify_rasters('swb_files/soil_group.dat', xll, yll, nrow, ncol, resolution)
arcify_rasters('swb_files/irrigation.dat', xll, yll, nrow, ncol, resolution)
arcify_rasters('swb_files/available_water_content.dat', xll, yll, nrow, ncol, resolution)


### write out the control file

In [ ]:
with open(ctl_filename, 'w') as ofp:
    ofp.write('# Freyberg Demonstration SWB synthetica MS Delta-ish\n\n')
    ofp.write('# grid info\n{0}\n\n'.format('-'*25))
    [ofp.write('{0}     {1}\n'.format(*i)) for i in grid_info.items()]    
    ofp.write('\n# methods info\n{0}\n\n'.format('-'*25))
    [ofp.write('{0}     {1}\n'.format(*i)) for i in methods.items()]
    ofp.write('\n# weather data info\n{0}\n\n'.format('-'*25))
    [ofp.write('{0}     {1}\n'.format(*i)) for i in weather_data.items()]
    ofp.write('\n# grids\n{0}\n\n'.format('-'*25))
    [ofp.write('{0}     {1}\n'.format(*i)) for i in grids.items()]
    ofp.write('\n# lookup tables\n{0}\n\n'.format('-'*25))
    [ofp.write('{0}     {1}\n'.format(*i)) for i in lookup_tables.items()]
    ofp.write('\n# general directives\n{0}\n\n'.format('-'*25))
    [ofp.write('{0}     {1}\n'.format(*i)) for i in general_directives.items()]
    